# Cleaning Filevine Contact Information

In [1]:
import numpy as np
import pandas as pd

In [6]:
# df_out = pd.read_excel('./data/List of Contacts - Referrals - Outbound.xlsx')
df_out = pd.read_parquet('../data/Referrals_App_Outbound.parquet')
df_out

,Project ID,Create Date,Date of Intake,Sign Up Date,Dr/Facility Referred To Person Id,Dr/Facility Referred To Full Name,Dr/Facility Referred To Address 1 Line 1,Dr/Facility Referred To Address 1 City,Dr/Facility Referred To Address 1 State,Dr/Facility Referred To Address 1 Zip,Dr/Facility Referred To's Details: Latitude,Dr/Facility Referred To's Details: Longitude,Dr/Facility Referred To Phone 1
0,991272125.0,2023-04-24 08:49:00,1970-01-01 00:00:00.000045040,1970-01-01 00:00:00.000045040,996605635,Bezak Chiropractic And Rehabilitation,7500 Hanover Parkway,Greenbelt,MD,20770,38.992916,-76.875824,(301) 220-0496 Main
1,991277077.0,2023-05-01 09:34:27,1970-01-01 00:00:00.000045047,1970-01-01 00:00:00.000045047,993726318,"Absolute Chiropractic Care - Oxon Hill, MD",5210 Indian Head Highway,Oxon Hill,MD,20745,38.818066,-76.999031,301-839-0500 Main
2,991273068.0,2023-05-03 06:29:12,1970-01-01 00:00:00.000045047,1970-01-01 00:00:00.000045049,993726318,"Absolute Chiropractic Care - Oxon Hill, MD",5210 Indian Head Highway,Oxon Hill,MD,20745,38.818066,-76.999031,301-839-0500 Main
3,991349758.0,2023-05-17 15:16:48,1970-01-01 00:00:00.000045051,1970-01-01 00:00:00.000045063,996605635,Bezak Chiropractic And Rehabilitation,7500 Hanover Parkway,Greenbelt,MD,20770,38.992916,-76.875824,(301) 220-0496 Main
4,991350809.0,2023-05-21 17:15:40,1970-01-01 00:00:00.000045061,1970-01-01 00:00:00.000045063,996701857,Waldorf Total Health Chiropractic & Physical T...,12102 Old Line Center,Waldorf,MD,20602,38.616638,-76.890701,(240) 754-7130 Work
...,...,...,...,...,...,...,...,...,...,...,...,...,...
357,992309632.0,2025-08-05 11:30:33,1970-01-01 00:00:00.000045874,1970-01-01 00:00:00.000045874,996576449,Pain and Rehab Center of Maryland - Camp Sprin...,5855 Allentown Road,Camp Springs,MD,20746,38.806683,-76.902763,(301) 925-2013 Work
358,992310813.0,2025-08-07 10:54:10,1970-01-01 00:00:00.000045876,1970-01-01 00:00:00.000045876,996701857,Waldorf Total Health Chiropractic & Physical T...,12102 Old Line Center,Waldorf,MD,20602,38.616638,-76.890701,(240) 754-7130 Work
359,992312808.0,2025-08-12 11:34:51,1970-01-01 00:00:00.000045881,1970-01-01 00:00:00.000045881,996605635,Bezak Chiropractic And Rehabilitation,7500 Hanover Parkway,Greenbelt,MD,20770,38.992916,-76.875824,(301) 220-0496 Main
360,992314325.0,2025-08-14 14:54:52,1970-01-01 00:00:00.000045883,1970-01-01 00:00:00.000045883,996613409,Effective Integrative Healthcare - Millersvill...,683 Old Mill Rd,Millersville,MD,21108,39.117565,-76.632660,410-928-4192 Personal Mobile


In [7]:
df_out.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 362 entries, 0 to 361
Data columns (total 13 columns):
 #   Column                                        Non-Null Count  Dtype         
---  ------                                        --------------  -----         
 0   Project ID                                    362 non-null    float64       
 1   Create Date                                   362 non-null    datetime64[ns]
 2   Date of Intake                                362 non-null    datetime64[ns]
 3   Sign Up Date                                  356 non-null    datetime64[ns]
 4   Dr/Facility Referred To Person Id             362 non-null    int64         
 5   Dr/Facility Referred To Full Name             362 non-null    object        
 6   Dr/Facility Referred To Address 1 Line 1      362 non-null    object        
 7   Dr/Facility Referred To Address 1 City        362 non-null    object        
 8   Dr/Facility Referred To Address 1 State       362 non-null    object  

In [8]:
str_cols = ['Dr/Facility Referred To Full Name',
            'Dr/Facility Referred To Address 1 Line 1',
            'Dr/Facility Referred To Address 1 City'
            ]

for col in str_cols:
    df_out[col] = (df_out[col]
                    .str.strip()
                    .str.title()
                    )

df_out['Dr/Facility Referred To Address 1 State'] = (df_out['Dr/Facility Referred To Address 1 State']
                                                        .str.strip()
                                                        .str.upper()
                                                        )

# df_out['Dr/Facility Referred To Address 1 Zip'] = df_out['Dr/Facility Referred To Address 1 Zip'].str.split('-')

df_out

,Project ID,Create Date,Date of Intake,Sign Up Date,Dr/Facility Referred To Person Id,Dr/Facility Referred To Full Name,Dr/Facility Referred To Address 1 Line 1,Dr/Facility Referred To Address 1 City,Dr/Facility Referred To Address 1 State,Dr/Facility Referred To Address 1 Zip,Dr/Facility Referred To's Details: Latitude,Dr/Facility Referred To's Details: Longitude,Dr/Facility Referred To Phone 1
0,991272125.0,2023-04-24 08:49:00,1970-01-01 00:00:00.000045040,1970-01-01 00:00:00.000045040,996605635,Bezak Chiropractic And Rehabilitation,7500 Hanover Parkway,Greenbelt,MD,20770,38.992916,-76.875824,(301) 220-0496 Main
1,991277077.0,2023-05-01 09:34:27,1970-01-01 00:00:00.000045047,1970-01-01 00:00:00.000045047,993726318,"Absolute Chiropractic Care - Oxon Hill, Md",5210 Indian Head Highway,Oxon Hill,MD,20745,38.818066,-76.999031,301-839-0500 Main
2,991273068.0,2023-05-03 06:29:12,1970-01-01 00:00:00.000045047,1970-01-01 00:00:00.000045049,993726318,"Absolute Chiropractic Care - Oxon Hill, Md",5210 Indian Head Highway,Oxon Hill,MD,20745,38.818066,-76.999031,301-839-0500 Main
3,991349758.0,2023-05-17 15:16:48,1970-01-01 00:00:00.000045051,1970-01-01 00:00:00.000045063,996605635,Bezak Chiropractic And Rehabilitation,7500 Hanover Parkway,Greenbelt,MD,20770,38.992916,-76.875824,(301) 220-0496 Main
4,991350809.0,2023-05-21 17:15:40,1970-01-01 00:00:00.000045061,1970-01-01 00:00:00.000045063,996701857,Waldorf Total Health Chiropractic & Physical T...,12102 Old Line Center,Waldorf,MD,20602,38.616638,-76.890701,(240) 754-7130 Work
...,...,...,...,...,...,...,...,...,...,...,...,...,...
357,992309632.0,2025-08-05 11:30:33,1970-01-01 00:00:00.000045874,1970-01-01 00:00:00.000045874,996576449,Pain And Rehab Center Of Maryland - Camp Sprin...,5855 Allentown Road,Camp Springs,MD,20746,38.806683,-76.902763,(301) 925-2013 Work
358,992310813.0,2025-08-07 10:54:10,1970-01-01 00:00:00.000045876,1970-01-01 00:00:00.000045876,996701857,Waldorf Total Health Chiropractic & Physical T...,12102 Old Line Center,Waldorf,MD,20602,38.616638,-76.890701,(240) 754-7130 Work
359,992312808.0,2025-08-12 11:34:51,1970-01-01 00:00:00.000045881,1970-01-01 00:00:00.000045881,996605635,Bezak Chiropractic And Rehabilitation,7500 Hanover Parkway,Greenbelt,MD,20770,38.992916,-76.875824,(301) 220-0496 Main
360,992314325.0,2025-08-14 14:54:52,1970-01-01 00:00:00.000045883,1970-01-01 00:00:00.000045883,996613409,Effective Integrative Healthcare - Millersvill...,683 Old Mill Rd,Millersville,MD,21108,39.117565,-76.632660,410-928-4192 Personal Mobile


In [9]:
full_address = ['Dr/Facility Referred To Person Id',
                'Dr/Facility Referred To Full Name',
                'Dr/Facility Referred To Address 1 Line 1',
                'Dr/Facility Referred To Address 1 City',
                'Dr/Facility Referred To Address 1 State',
                'Dr/Facility Referred To Address 1 Zip',
                "Dr/Facility Referred To's Details: Latitude",
                "Dr/Facility Referred To's Details: Longitude",
                "Dr/Facility Referred To Phone 1"
                ]

df_out_gb = (df_out.groupby(full_address, as_index=False, sort = False)['Project ID']
            .count()
            .rename(columns = {'Project ID':'Referral Count'})
            .sort_values(by = 'Referral Count', ascending = False)
            )
df_out_gb

,Dr/Facility Referred To Person Id,Dr/Facility Referred To Full Name,Dr/Facility Referred To Address 1 Line 1,Dr/Facility Referred To Address 1 City,Dr/Facility Referred To Address 1 State,Dr/Facility Referred To Address 1 Zip,Dr/Facility Referred To's Details: Latitude,Dr/Facility Referred To's Details: Longitude,Dr/Facility Referred To Phone 1,Referral Count
2,996701857,Waldorf Total Health Chiropractic & Physical T...,12102 Old Line Center,Waldorf,MD,20602,38.616638,-76.890701,(240) 754-7130 Work,60
0,996605635,Bezak Chiropractic And Rehabilitation,7500 Hanover Parkway,Greenbelt,MD,20770,38.992916,-76.875824,(301) 220-0496 Main,38
6,996356411,Dunkirk Chiropractic & Wellness Center,10020 Southern Maryland Blvd,Dunkirk,MD,20754,38.714687,-76.659378,(410) 286-3335 Work,24
3,996605523,"Effective Integrative Healthcare - Lanham, Md",7400 Riverdale Rd,Lanham,MD,20706,38.962082,-76.884651,(301) 960-2285 Work,21
16,996616309,Rebound Integrative Health & Sports Clinic - G...,7701 Greenbelt Rd,Greenbelt,MD,20770-6521,38.994034,-76.874779,(240) 261-7229 Fax,18
24,996701795,"Ameriwell Chiropractic - Silver Spring, Md",804 Pershing Dr,Silver Spring,MD,20910,38.997936,-77.023117,(301) 587-4000 Work,16
25,996700633,Kaizo Health Chiropractic & Rehabilitation - L...,4301 Garden City Dr,Landover,MD,20785,38.948257,-76.868942,(301) 577-1115,14
20,996605561,Kaizo Health Chiropractic & Rehabilitation - F...,9300 Livingston Rd,Fort Washington,MD,20744,38.764030,-76.995247,(301) 203-6734 Work,13
26,993878578,"Maryland Healthcare Clinics - Baltimore, Md",6615 Reisterstown Road,Baltimore,MD,21215-2633,39.357426,-76.703949,(888) 570-0088 Work,13
9,994689737,"Maryland Healthcare Clinics - Washington, Dc Ne",1647 Benning Road,Washington,DC,20002,38.894836,-76.995033,(888) 570-0088 Work,12


In [10]:
new_col_names = {
    'Dr/Facility Referred To Person Id': 'Person ID',
    'Dr/Facility Referred To Full Name': 'Full Name',
    'Dr/Facility Referred To Address 1 Line 1': 'Street',
    'Dr/Facility Referred To Address 1 City': 'City',
    'Dr/Facility Referred To Address 1 State': 'State',
    'Dr/Facility Referred To Address 1 Zip':'Zip',
    "Dr/Facility Referred To's Details: Latitude":'Latitude',
    "Dr/Facility Referred To's Details: Longitude":'Longitude',
    'Dr/Facility Referred To Phone 1':'Phone Number',
    'Project ID': 'Referral Count'
}

df_out_gb = df_out_gb.rename(columns = new_col_names)
df_out_gb

,Person ID,Full Name,Street,City,State,Zip,Latitude,Longitude,Phone Number,Referral Count
2,996701857,Waldorf Total Health Chiropractic & Physical T...,12102 Old Line Center,Waldorf,MD,20602,38.616638,-76.890701,(240) 754-7130 Work,60
0,996605635,Bezak Chiropractic And Rehabilitation,7500 Hanover Parkway,Greenbelt,MD,20770,38.992916,-76.875824,(301) 220-0496 Main,38
6,996356411,Dunkirk Chiropractic & Wellness Center,10020 Southern Maryland Blvd,Dunkirk,MD,20754,38.714687,-76.659378,(410) 286-3335 Work,24
3,996605523,"Effective Integrative Healthcare - Lanham, Md",7400 Riverdale Rd,Lanham,MD,20706,38.962082,-76.884651,(301) 960-2285 Work,21
16,996616309,Rebound Integrative Health & Sports Clinic - G...,7701 Greenbelt Rd,Greenbelt,MD,20770-6521,38.994034,-76.874779,(240) 261-7229 Fax,18
24,996701795,"Ameriwell Chiropractic - Silver Spring, Md",804 Pershing Dr,Silver Spring,MD,20910,38.997936,-77.023117,(301) 587-4000 Work,16
25,996700633,Kaizo Health Chiropractic & Rehabilitation - L...,4301 Garden City Dr,Landover,MD,20785,38.948257,-76.868942,(301) 577-1115,14
20,996605561,Kaizo Health Chiropractic & Rehabilitation - F...,9300 Livingston Rd,Fort Washington,MD,20744,38.764030,-76.995247,(301) 203-6734 Work,13
26,993878578,"Maryland Healthcare Clinics - Baltimore, Md",6615 Reisterstown Road,Baltimore,MD,21215-2633,39.357426,-76.703949,(888) 570-0088 Work,13
9,994689737,"Maryland Healthcare Clinics - Washington, Dc Ne",1647 Benning Road,Washington,DC,20002,38.894836,-76.995033,(888) 570-0088 Work,12


In [11]:
df_out_gb['Zip'] = df_out_gb['Zip'].astype('str')

In [ ]:
# df_out_gb.to_csv('./data/cleaned_outbound_referrals.csv', index = False)
# df_out_gb.to_excel('../data/cleaned_outbound_referrals.xlsx', index = False)

df_out_gb.to_parquet('../data/cleaned_outbound_referrals.parquet', index = False)
# df_out_gb.to_feather('./data/cleaned_outbound_referrals.feather')

In [ ]:
# pd.read_parquet('./data/cleaned_outbound_referrals.parquet')

# Cleaning Inbound Referrals

In [ ]:
df_in = pd.read_excel('./data/List of Contacts - Referrals - Inbound.xlsx')
df_in

In [ ]:
df_in.info()

In [ ]:
str_cols = ['Referred From Full Name',
            'Referred From Address 1 Line 1',
            'Referred From Address 1 City',
            'Secondary Referred From Full Name',
            'Secondary Referred From Address 1 Line 1',
            'Secondary Referred From Address 1 City',
            ]

for col in str_cols:
    df_in[col] = (df_in[col]
                    .str.strip()
                    .str.title()
                    )

state_cols = ['Referred From Address 1 State',
              'Secondary Referred From Address 1 State']

for col in state_cols:
    df_in[col] = (df_in[col]
                    .str.strip()
                    .str.upper()
                    )

# df_out['Dr/Facility Referred To Address 1 Zip'] = df_out['Dr/Facility Referred To Address 1 Zip'].str.split('-')

df_in

In [ ]:
new_col_names = {
    'Referral Source': 'Referral Source',
    'Referred From Person Id':'Person Id',
    'Referred From Full Name':'Full Name',
    'Referred From Address 1 Line 1':'Street',
    'Referred From Address 1 City':'City',
    'Referred From Address 1 State':'State',
    'Referred From Address 1 Zip':'Zip'
}

# df_in_rf2.rename(columns=new_col_names)

df_in = df_in.rename(columns=new_col_names)
df_in

In [ ]:
shared_cols = ['Project ID',
                'Create Date',
                'Date of Intake',
                'Sign Up Date'
                ]

ref1_cols = ['Referral Source',
            'Person Id',
            'Full Name',
            'Street',
            'City',
            'State',
            'Zip']

ref2_cols = ['Secondary Referral Source',
            'Secondary Referred From Person Id',
            'Secondary Referred From Full Name',
            'Secondary Referred From Address 1 Line 1',
            'Secondary Referred From Address 1 City',
            'Secondary Referred From Address 1 State',
            'Secondary Referred From Address 1 Zip'] 

df_in_rf2 = df_in[shared_cols + ref2_cols]

df_in_rf2 = df_in_rf2[df_in_rf2['Secondary Referred From Person Id'] > 0]
df_in_rf2.head(5)

In [ ]:
df_in_rf1 = df_in.drop(columns = ref2_cols)
df_in_rf1

In [ ]:
new_col_names = {
    'Secondary Referral Source': 'Referral Source',
    'Secondary Referred From Person Id':'Person Id',
    'Secondary Referred From Full Name':'Full Name',
    'Secondary Referred From Address 1 Line 1':'Street',
    'Secondary Referred From Address 1 City':'City',
    'Secondary Referred From Address 1 State':'State',
    'Secondary Referred From Address 1 Zip':'Zip'
}

# df_in_rf2.rename(columns=new_col_names)

df_in_rf2 = df_in_rf2.rename(columns=new_col_names)
df_in_rf2

In [ ]:
df_in_full = pd.concat([df_in_rf1, df_in_rf2], axis = 0, ignore_index=True)
df_in_full

In [ ]:
df_in_full = df_in_full.dropna(subset = ['Referral Source','Person Id'], how = 'any')

df_in_full = (df_in_full[df_in_full['Referral Source'].str.contains('Doctor')]).reset_index(drop = True)

df_in_full

In [ ]:
df_in_full['Referral Source'].value_counts()

In [ ]:
full_address = ['Person Id',
                'Full Name',
                'Street',
                'City',
                'State',
                'Zip'
                ]

df_in_gb = (df_in_full.groupby(full_address, as_index=False)['Project ID']
            .count()
            .rename(columns = {'Project ID':'Referral Count'})
            .sort_values(by = 'Referral Count', ascending = False)
            )
df_in_gb

In [ ]:
df_in_gb.to_excel('./data/cleaned_inbound_referrals.xlsx', index = False)